In [59]:
import torch
import json
import os
from pathlib import Path
import numpy as np

In [ ]:
class Env:
    def __init__(self, envFile):
        self.file_path = Path(envFile)
        with open(envFile) as f:
            self.data = json.load(f)
        self.parseMap()
        self.parseAgents()
        self.parseTasks()
        self.numTasksReveal = int(self.data["numTasksReveal"])
        self.taskAssignmentStrategy = self.data["taskAssignmentStrategy"]

    def parseMap(self):
        with open(self.file_path.parent/self.data["mapFile"]) as f:
            lines = f.readlines()
        type_map = lines[0].strip().split(" ")
        height = int(lines[1].strip().split(" ")[1])
        width = int(lines[2].strip().split(" ")[1])
        map_str = lines[3]
        self.map_1darr = []
        for line in lines[4:]:
            for char in line.strip():
                self.map_1darr.append(char)
        
    def parseAgents(self):
        with open(self.file_path.parent/self.data["agentFile"]) as f:
            lines = f.readlines()
        self.n_agents = int(lines[0].strip().split(" ")[0])
        self.agents = []
        for line in lines[1:]:
            self.agents.append(int(line.strip().split(" ")[0]))
        self.agents = np.array(self.agents)

    def parseTasks(self):
        with open(self.file_path.parent/self.data["taskFile"]) as f:
            lines = f.readlines()
        self.n_tasks = int(lines[0].strip().split(" ")[0])
        self.tasks = []
        for line in lines[1:]:
            self.tasks.append(int(line.strip().split(" ")[0]))
        self.tasks = np.array(self.tasks)

class BatchedEnvironment:
    def __init__(self, environments):
        self.maps = torch.stack([torch.tensor(env.map, dtype=torch.float32) for env in environments])
        self.curr_states = torch.stack([torch.tensor(env.agents) for env in environments])
        self.goal_locations = torch.stack([torch.tensor(env.tasks) for env in environments])


In [55]:
p = ["/home/aarav/aaravpandya/example_problems/city.domain/paris_200.json", "/home/aarav/aaravpandya/example_problems/game.domain/brc202d_200.json"]
envs = []
for e in p:
    envs.append(Env(e))

(200,)